## Q-Clustering

In [68]:
import numpy as np
from scipy.spatial import distance
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D



##########################################

##########################################



def VGradient(data:np.ndarray,sigma,x:np.ndarray=None,coeffs:np.ndarray=None):
    """Compute the value of the qunatum potential generated by points data, and its gradient, at point x.
    *inputs:*
    data: a n-by-q numpy.ndarray with samples of the Parzen function (quantum wave function) Each row is a sample.
    sigma: a numeric scalar representing the width the Gaussian around each sample
    x: a m-by-1 numpy.ndarray of points where the values of the potential and the gradients are to be computed. each row is a point. if x is not given, data will be used instead.
    coeffs: (optional) a n-by-1 numpy.ndarray of weights to each sample in data
    *outputs:*
    v: a m-by-1 numpy.ndarray of the values of the quantum potential function, for each point in x
    dv: a m-by-q numpy.ndarray of the gradients of the quantum potential, for each point in x"""
    
    if x is None:
        x = data.copy()
    
    if coeffs is None:
        coeffs = np.ones((data.shape[0],))
    
        
    twoSigmaSquared = 2*sigma**2
        
    data = data[np.newaxis,:,:]
    x = x[:,np.newaxis,:]
    differences = x-data
    squaredDifferences = np.sum(np.square(differences),axis=2)
    gaussian = np.exp(-(1/twoSigmaSquared)*squaredDifferences)
    laplacian = np.sum(coeffs*gaussian*squaredDifferences,axis=1)
    parzen = np.sum(coeffs*gaussian,axis=1)
    v = 1 + (1/twoSigmaSquared)*laplacian/parzen

    dv = -1*(1/parzen[:,np.newaxis])*np.sum(differences*((coeffs*gaussian)[:,:,np.newaxis])*(twoSigmaSquared*(v[:,np.newaxis,np.newaxis])-(squaredDifferences[:,:,np.newaxis])),axis=1)
    
    v = v-1
    
    return v, dv

def SGradient(data:np.ndarray,sigma,x:np.ndarray=None,coeffs:np.ndarray=None):
    """Compute the value of the entropy generated by points data, and its gradient, at point x.
    *inputs:*
    data: a n-by-q numpy.ndarray with samples of the Parzen function. Each row is a sample.
    sigma: a numeric scalar representing the width the Gaussian around each sample
    x: a m-by-1 numpy.ndarray of points where the values of the entropy and the gradients are to be computed. each row is a point. if x is not given, data will be used instead.
    coeffs: (optional) a n-by-1 numpy.ndarray of weights to each sample in data
    *outputs:*
    s: a m-by-1 numpy.ndarray of the values of the entropy, for each point in x
    ds: a m-by-q numpy.ndarray of the gradients of the entropy, for each point in x"""
    if x is None:
        x = data.copy()
        
    if coeffs is None:
        coeffs = np.ones((data.shape[0],))
    
    twoSigmaSquared = 2 * sigma ** 2
    
    data = data[np.newaxis, :, :]
    x = x[:, np.newaxis, :]
    differences = x - data
    squaredDifferences = np.sum(np.square(differences), axis=2)
    gaussian = np.exp(-(1 / twoSigmaSquared) * squaredDifferences)
    laplacian = np.sum(coeffs*gaussian * squaredDifferences, axis=1)
    parzen = np.sum(coeffs*gaussian, axis=1)
    v = (1 / twoSigmaSquared) * laplacian / parzen
    s = v + np.log(np.abs(parzen))
    
    ds = (1 / parzen[:, np.newaxis]) * np.sum(differences * ((coeffs*gaussian)[:, :, np.newaxis]) * (
    twoSigmaSquared * (v[:, np.newaxis, np.newaxis]) - (squaredDifferences[:, :, np.newaxis])), axis=1)
    
    return s, ds

def PGradient(data:np.ndarray,sigma,x:np.ndarray=None,coeffs:np.ndarray=None):
    """Compute the value of the Parzen function generated by points data, and its gradient, at point x.
        *inputs:*
        data: a n-by-q numpy.ndarray with samples of the Parzen function. Each row is a sample.
        sigma: a numeric scalar representing the width the Gaussian around each sample
        x: a m-by-1 numpy.ndarray of points where the values of the Parzen and the gradients are to be computed. each row is a point. if x is not given, data will be used instead.
        coeffs: (optional) a n-by-1 numpy.ndarray of weights to each sample in data
        *outputs:*
        p: a m-by-1 numpy.ndarray of the values of the Parzen function, for each point in x
        dp: a m-by-q numpy.ndarray of the gradients of the Parzen function, for each point in x"""
    if x is None:
        x = data.copy()
        
    if coeffs is None:
        coeffs = np.ones((data.shape[0],))
    
    twoSigmaSquared = 2 * sigma ** 2
    
    data = data[np.newaxis, :, :]
    x = x[:, np.newaxis, :]
    differences = x - data
    squaredDifferences = np.sum(np.square(differences), axis=2)
    gaussian = np.exp(-(1 / twoSigmaSquared) * squaredDifferences)
    p = np.sum(coeffs*gaussian,axis=1)
    
    dp = -1*np.sum(differences * ((coeffs*gaussian)[:, :, np.newaxis]) * twoSigmaSquared,axis=1)
    
    return p, dp

def getApproximateParzen(data:np.ndarray,sigma,voxelSize):
    """compute samples of the approximate Parzen functon, and their weights
    *inputs:*
        data: a n-by-q numpy.ndarray with samples of the Parzen function. Each row is a sample.
        sigma:  a numeric scalar representing the width the Gaussian around each sample.
        voxelSize: size of the side of a (hyper-)voxel in q dimensions, such that each voxel will contain at most one data point.
    *outputs:*
        newData: a m-by-q numpy.ndarray with the new data points, at most one per voxel
        coeffs: a m-by-q numpy.ndarray with the weights of each new data point"""
    newData = uniqueRows(np.floor(data/voxelSize)*voxelSize+voxelSize/2)[0]
    
    nMat = np.exp(-1*distance.squareform(np.square(distance.pdist(newData)))/(4*sigma**2))
    mMat = np.exp(-1 * np.square(distance.cdist(newData,data)) / (4 * sigma ** 2))
    cMat = np.linalg.solve(nMat,mMat)
    coeffs = np.sum(cMat,axis=1)
    coeffs = data.shape[0]*coeffs/sum(coeffs)
    
    return newData,coeffs



#######################################################3


def uniqueRows(x):
    """return the unique rows of x, their indexes, the reverse indexes and the counts"""
    y = np.ascontiguousarray(x).view(np.dtype((np.void, x.dtype.itemsize * x.shape[1])))
    _, inds,indsInverse,counts = np.unique(y, return_index=True,return_inverse=True,return_counts=True)

    xUnique = x[inds]
    return xUnique,inds,indsInverse,counts




######################################################


def GradientDescent(data,sigma,repetitions=1,stepSize=None,clusteringType='v',recalculate=False,returnHistory=False,stopCondition=True,voxelSize=None):
    """"""
    
    n = data.shape[0]

    useApproximation = (voxelSize is not None)
    
    if stepSize is None:
        stepSize = sigma/10
    
    if clusteringType == 'v':
        gradientFunction = VGradient
    elif clusteringType == 's':
        gradientFunction = SGradient
    else:
        gradientFunction = PGradient

    if useApproximation:
        newData, coeffs = getApproximateParzen(data, sigma, voxelSize)
    else:
        coeffs = None

    if recalculate:
        if useApproximation:
            x = np.vstack((data,newData))
            data = x[data.shape[0]:]
        else:
            x = data
    else:
        if useApproximation:
            x = data
            data = newData
        else:
            x = data.copy()
        
        
    if returnHistory:
        xHistory = np.zeros((n,x.shape[1],repetitions+1))
        xHistory[:,:,0] = x[:n,:].copy()
        
    if stopCondition:
        prevX = x[:n].copy()

    for i in range(repetitions):
        if ((i>0) and (i%10==0)):
            if stopCondition:
                if np.all(np.linalg.norm(x[:n]-prevX,axis=1) < np.sqrt(3*stepSize**2)):
                    i = i-1
                    break
                prevX = x[:n].copy()
            
        f,df = gradientFunction(data,sigma,x,coeffs)
        df = df/np.linalg.norm(df,axis=1)[:,np.newaxis]
        x[:] = x + stepSize*df

        if returnHistory:
            xHistory[:, :, i+1] = x[:n].copy()
            
    x = x[:n]

    if returnHistory:
        xHistory = xHistory[:,:,:(i+2)]
        return x,xHistory
    else:
        return x

def PerformFinalClustering(data,stepSize):
    """"""
    clusters = np.zeros((data.shape[0]))
    i = np.array([0])
    c = 0
    distances = distance.squareform(distance.pdist(data))
    while i.shape[0]>0:
        i = i[0]
        inds = np.argwhere(clusters==0)
        clusters[inds[distances[i,inds] <= 3*stepSize]] = c
        c += 1
        i = np.argwhere(clusters==0)
    return clusters


##############################################

def displayClustering(xHistory,clusters=None):
    """"""
    plt.ion()
    plt.figure(figsize=(20, 12))
    if clusters is None:
        clusters = np.zeros((xHistory.shape[0],))
    if xHistory.shape[1] == 1:
        plt.axes(aspect='equal')
        sc = plt.scatter(xHistory[:,:,0],xHistory[:,:,0]*0,c=clusters,s=10)
        plt.xlim((np.min(xHistory),np.max(xHistory)))
        plt.ylim((-1,1))
        for i in range(xHistory.shape[2]):
            sc.set_offsets(xHistory[:, :, i])
            plt.title('step #' + str(i) + '/' + str(xHistory.shape[2]-1))
            plt.pause(0.05)
    elif xHistory.shape[1] == 2:
        plt.axes(aspect='equal')
        sc = plt.scatter(xHistory[:, 0, 0], xHistory[:, 1, 0] , c=clusters, s=20)
        plt.xlim((np.min(xHistory[:,0,:]), np.max(xHistory[:,0,:])))
        plt.ylim((np.min(xHistory[:, 1, :]), np.max(xHistory[:, 1, :])))
        for i in range(xHistory.shape[2]):
            sc.set_offsets(xHistory[:, :, i])
            plt.title('step #' + str(i) + '/' + str(xHistory.shape[2]-1))
            plt.pause(0.2)
    else:
        if xHistory.shape[1] > 3:
            pca = PCA(3)
            pca.fit(xHistory[:,:,0])
            newXHistory = np.zeros((xHistory.shape[0],3,xHistory.shape[2]))
            for i in range(xHistory.shape[2]):
                newXHistory[:,:,i] = pca.transform(xHistory[:,:,i])
            xHistory = newXHistory

        ax = plt.axes(aspect='equal',projection='3d')
        sc = ax.scatter(xHistory[:, 0, 0], xHistory[:, 1, 0],xHistory[:, 2, 0], c=clusters, s=20)
        ax.set_xlim((np.min(xHistory[:, 0, :]), np.max(xHistory[:, 0, :])))
        ax.set_ylim((np.min(xHistory[:, 1, :]), np.max(xHistory[:, 1, :])))
        ax.set_zlim((np.min(xHistory[:, 2, :]), np.max(xHistory[:, 2, :])))
        for i in range(xHistory.shape[2]):
            sc._offsets3d =  (np.ravel(xHistory[:, 0, i]),np.ravel(xHistory[:, 1, i]),np.ravel(xHistory[:, 2, i]))
            plt.gcf().suptitle('step #' + str(i) + '/' + str(xHistory.shape[2]-1))
            plt.pause(0.01)

    plt.ioff()
    plt.close()

In [2]:
from sklearn.datasets import load_iris
data_iris = load_iris()

In [3]:
#iris_target = data_iris.target

### Replace this with dtm for CFPB data 

import os 
import string
import pandas as pd 

#set working environment 
os.chdir("/home/spenser/Downloads/case_study")

#Import Data
CFPB_1 = pd.read_csv("cfpb_1.csv")

CFPB_2 = pd.read_csv("cfpb_2.csv", header = None)
CFPB_2.columns = ['complaint_id', 'text']

CFPB_3 = pd.read_csv("cfpb_3.csv", header = None)
CFPB_3.columns = ['complaint_id', 'text']

CFPB_4 = pd.read_csv("cfpb_4.csv", header = None)
CFPB_4.columns = ['complaint_id', 'text']

CFPB_5 = pd.read_csv("cfpb_5.csv", header = None)
CFPB_5.columns = ['complaint_id', 'text']

CFPB_text = pd.concat([CFPB_1, CFPB_2, CFPB_3, CFPB_4, CFPB_5])

file_no_text = pd.read_csv("cfpb_triage_case_study_notext.csv")

# Merging Case Study Files to Complaint IDs
CFPB_Case_Study_Joined = file_no_text.merge(CFPB_text, on = 'complaint_id', how ='left')


CFPB_Case_Study_Joined["text_lower"] = CFPB_Case_Study_Joined["text"].str.lower()

CFPB_Case_Study_Joined["text_lower"] = CFPB_Case_Study_Joined["text_lower"].str.replace(r'\nRevision: (\d+)\n', '') #remove digits

def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    return text

CFPB_Case_Study_Joined["text_clean"] = CFPB_Case_Study_Joined['text_lower'].apply(remove_punctuations)  #remove punctuation

#Adding this due to finding below that pre-cleansed text is corrupt. (contains cases like can t) . Remove single stand-alone characters. ("a", "e", etc)

CFPB_Case_Study_Joined["text_clean"] = CFPB_Case_Study_Joined["text_clean"].str.replace(r'\b(?<=)[a-z](?=)\b', '') #remove single stand-alone characters.



###############

import gc

# Remove stop words.

# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

CFPB_Case_Study_Joined['text_clean_stopwords'] = CFPB_Case_Study_Joined['text_clean'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)])) #stop word removal.

In [66]:
#### Due to memory errors, using doc2vec approach. This will reduce dimensionality. I am not sure at this time if this
### is appropriate for the quantum clustering methodology. 

import nltk
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

max_epochs = 10
vec_size = 300
alpha = 0.025


def tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text):
        for word in nltk.word_tokenize(sent):
            if len(word) < 2:
                continue
            tokens.append(word.lower())
    return tokens
train_tagged = CFPB_Case_Study_Joined.apply(
    lambda r: TaggedDocument(words=tokenize_text(r['text_clean_stopwords']), tags=[r.product_group]), axis=1)

    
NUM_VECTORS = 50

vector_count = NUM_VECTORS
model = Doc2Vec(size=vector_count, min_count=1, dm=0, iter=1,
                dm_mean=1, dbow_words=1, workers=12)
model.build_vocab(train_tagged)

for epoch in range(max_epochs):
    print ("Training epoch", epoch)
    model.train(train_tagged,  total_examples=model.corpus_count,  epochs=model.epochs)
    model.alpha -= 0.02
    model.min_alpha = model.alpha

Training epoch 0
Training epoch 1
Training epoch 2
Training epoch 3
Training epoch 4
Training epoch 5
Training epoch 6
Training epoch 7
Training epoch 8
Training epoch 9


In [67]:
def vec_for_learning(model, tagged_docs):
    sents = tagged_docs.values
    targets, regressors = zip(*[(doc.tags[0], model.infer_vector(doc.words, 0.03, steps=20)) for doc in sents])
    return targets, regressors


y_train, X_train = vec_for_learning(model, train_tagged)

In [59]:
X_train_pd = pd.DataFrame(X_train)

In [24]:
#from sklearn.feature_extraction.text import CountVectorizer
#from nltk.tokenize import RegexpTokenizer

#CFPB_Subset = CFPB_Case_Study_Joined[0:1000]


##tokenizer to remove unwanted elements from out data like symbols and numbers
#token = RegexpTokenizer(r'[a-zA-Z0-9]+')
#cv = CountVectorizer(lowercase=True,stop_words='english', ngram_range = (1,1),tokenizer = token.tokenize)
#text_counts= cv.fit_transform(CFPB_Subset["text_clean_stopwords"])

In [25]:
text_counts_array = text_counts.toarray()

In [46]:
text_counts

<1000x6919 sparse matrix of type '<class 'numpy.int64'>'
	with 59322 stored elements in Compressed Sparse Row format>

In [63]:
# Standardize the data attributes for the Iris dataset.
from sklearn.datasets import load_iris
from sklearn import preprocessing
# load the Iris dataset


# separate the data and target attributes
X = np.array(X_train)
# standardize the data attributes
#standardized_X = preprocessing.scale(text_counts_array)
#X.shape

In [64]:
X.shape

(268701, 300)

In [42]:
data_iris.data

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1],
       [5.4, 3.7, 1.5, 0.2],
       [4.8, 3.4, 1.6, 0.2],
       [4.8, 3. , 1.4, 0.1],
       [4.3, 3. , 1.1, 0.1],
       [5.8, 4. , 1.2, 0.2],
       [5.7, 4.4, 1.5, 0.4],
       [5.4, 3.9, 1.3, 0.4],
       [5.1, 3.5, 1.4, 0.3],
       [5.7, 3.8, 1.7, 0.3],
       [5.1, 3.8, 1.5, 0.3],
       [5.4, 3.4, 1.7, 0.2],
       [5.1, 3.7, 1.5, 0.4],
       [4.6, 3.6, 1. , 0.2],
       [5.1, 3.3, 1.7, 0.5],
       [4.8, 3.4, 1.9, 0.2],
       [5. , 3. , 1.6, 0.2],
       [5. , 3.4, 1.6, 0.4],
       [5.2, 3.5, 1.5, 0.2],
       [5.2, 3.4, 1.4, 0.2],
       [4.7, 3.2, 1.6, 0.2],
       [4.8, 3.1, 1.6, 0.2],
       [5.4, 3.4, 1.5, 0.4],
       [5.2, 4.1, 1.5, 0.1],
       [5.5, 4.2, 1.4, 0.2],
       [4.9, 3

In [65]:
import numpy as np
#import cluster


sigma=0.55
repetitions=100
stepSize=0.1
clusteringType='v'
recalculate=False
returnHistory=True
stopCondition=True
voxelSize = None

x,xHistory = GradientDescent(X,sigma=sigma,repetitions=repetitions,stepSize=stepSize,clusteringType=clusteringType,recalculate=recalculate,returnHistory=returnHistory,stopCondition=stopCondition,voxelSize=voxelSize)

clusters = PerformFinalClustering(x,stepSize)

displayClustering(xHistory,clusters)

MemoryError: 

In [45]:
coeffs = getApproximateParzen(data_iris.data, sigma, voxelSize)
f,df = VGradient(data_iris.data,sigma,x,coeffs)

TypeError: unsupported operand type(s) for /: 'float' and 'NoneType'